# Final Assignment

In this assignment, you will build a relation extraction model for scientific articles based on the [ScienceIE dataset](https://scienceie.github.io/) in a group of up to 3 students. This is the same dataset that was used for Assignment 2, where you had to train a model to extract keyphrases. You are welcome to build on code any team member already wrote for Assignment 2.

You will build and train relation extraction models on the ScienceIE dataset. For this, you will also need to do data preprocessing to convert the ScienceIE data into a format suitable for training a relation extraction models. 

Your mark will depend on:

* your **reasoning behind modelling choices** made
* the correct **implementations** of your relation extraction models, and
* the **performance** of your models on a held-out test set.

To develop your model you have access to:

* The data in `data/scienceie/`. Remember to un-tar the data.tar.gz file.
* Libraries on the [docker image](https://cloud.docker.com/repository/docker/bjerva/stat-nlp-book) which contains everything in [this image](https://github.com/jupyter/docker-stacks/tree/master/scipy-notebook), including scikit-learn, torch 1.2.0 and tensorflow 1.14.0. 


As with the previous assignment, since we have to run the notebooks of all students, and because writing efficient code is important, your notebook should run in 10 minutes at most, including package loading time, on your machine.
Furthermore, you are welcome to provide a saved version of your model with loading code. In this case loading, testing, and evaluation has to be done in 10 minutes. You can use the dev set to check if this is the case, and assume that it will be fine for the held-out test set if so.

## Setup Instructions
It is important that this file is placed in the **correct directory**. It will not run otherwise. The correct directory is

    DIRECTORY_OF_YOUR_BOOK/assignments/2019/final_assignment/problem/
    
where `DIRECTORY_OF_YOUR_BOOK` is a placeholder for the directory you downloaded the book to. After you placed it there, **rename the file** to your UCPH ID (of the form `xxxxxx`). 

## General Instructions
This notebook will be used by you to provide your solution, and by us to both assess your solution and enter your marks. It contains three types of sections:

1. **Setup** Sections: these sections set up code and resources for assessment. **Do not edit these**. 
2. **Assessment** Sections: these sections are used for both evaluating the output of your code, and for markers to enter their marks. **Do not edit these**. 
3. **Task** Sections: these sections require your solutions. They may contain stub code, and you are expected to edit this code. For free text answers simply edit the markdown field.  

Note that you are free to **create additional notebook cells** within a task section. 

**Do not share** this assignment publicly, by uploading it online, emailing it to friends etc. 

**Do not** copy code from the Web or from other students, this will count as plagiarism.

## Submission Instructions

To submit your solution:

* Make sure that your solution is fully contained in this notebook. 
* **Rename this notebook to your UCPH ID** (of the form "xxxxxx"), if you have not already done so.
* Download the notebook in Jupyter via *File -> Download as -> Notebook (.ipynb)*.
* Upload the notebook to Absalon.


## <font color='green'>Setup 1</font>: Load Libraries
This cell loads libraries important for evaluation and assessment of your model. **Do not change it.**

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#! SETUP 1
import sys, os
_snlp_book_dir = "../../../../"
sys.path.append(_snlp_book_dir) 
import math
from glob import glob
from os.path import isfile, join
from statnlpbook.vocab import Vocab
from statnlpbook.scienceie import calculateMeasures
import shutil
import string

## <font color='blue'>Task 1</font>: Convert dataset between standoff and IOB format

We want to work with [the ScienceIE dataset](https://scienceie.github.io) that can be found in the `data/scienceie/` directory of the repository.  This dataset comes with **standoff annotation** for keyphrases and relations between them.  This means that for each document in the dataset, there are two files: a `.txt` file with the raw sentences, and a `.ann` file with the annotated keyphrases.  

For example, this is one of the `.txt` files from the training set:

```
Failure of structural components is a major concern in the nuclear power industry and represents not only a safety issue, but also a hazard to economic performance. Stress corrosion cracking (SCC), and especially intergranular stress corrosion cracking (IGSCC), have proved to be a significant potential cause of failures in the nuclear industry in materials such as Alloy 600 (74% Ni, 16% Cr and 8% Fe) and stainless steels, especially in Pressurised Water Reactors (PWR) [1–5]. Stress corrosion cracking in pressurized water reactors (PWSCC) occurs in Alloy 600 in safety critical components, such as steam generator tubes, heater sleeves, pressurized instrument penetrations and control rod drive mechanisms [2,6,7]. Understanding the mechanisms that control SCC in this alloy will allow for continued extensions of life in current plant as well as safer designs of future nuclear reactors.
```

And this is the corresponding `.ann` file:

```
T1	Material 11 32	structural components
T2	Process 0 32	Failure of structural components
T3	Process 254 259	IGSCC
T4	Process 213 252	intergranular stress corrosion cracking
*	Synonym-of T4 T3
T5	Process 165 190	Stress corrosion cracking
T6	Process 192 195	SCC
*	Synonym-of T5 T6
T7	Material 367 376	Alloy 600
T8	Material 378 402	74% Ni, 16% Cr and 8% Fe
*	Synonym-of T7 T8
T9	Material 408 424	stainless steels
T10	Material 440 466	Pressurised Water Reactors
T11	Material 468 471	PWR
T12	Process 480 505	Stress corrosion cracking
T13	Material 509 535	pressurized water reactors
T14	Material 537 542	PWSCC
*	Synonym-of T13 T14
T15	Material 554 563	Alloy 600
T16	Material 603 624	steam generator tubes
T17	Material 626 640	heater sleeves
T18	Material 642 677	pressurized instrument penetrations
T19	Material 682 710	control rod drive mechanisms
T20	Material 762 765	SCC
T21	Material 774 779	alloy
T22	Material 835 840	plant
T23	Task 852 892	safer designs of future nuclear reactors
T24	Material 876 892	nuclear reactors
T25	Material 567 593	safety critical components
R1	Hyponym-of Arg1:T16 Arg2:T25	
R2	Hyponym-of Arg1:T17 Arg2:T25	
R3	Hyponym-of Arg1:T18 Arg2:T25	
R4	Hyponym-of Arg1:T19 Arg2:T25
```

Note: Besides keyphrases, which you are already familiar with from Assignment 2, the `.ann` files also contain relation annotations labeled `Hyponym-of` and `Synonym-of`. These are relations between keyphrases. 

`Synonym-of` is an undirected relation, meaning that if you see a line like this:

```*	Synonym-of T13 T14```

The order of keyphrases could be swapped, i.e. the following would also hold:

```*	Synonym-of T14 T13```

The evaluation script will thus be agnostic to the order in which the keyphrases between which `Synonym-of` relations hold are ordered.

`Hyponym-of`, on the other hand, is a directed relation, meaning that it is order-sensitive, and that the evaluation script will take the order of keyphrases between which `Hyponym-of` relations hold into account.

The `.ann` standoff format is **documented in [the brat documentation](http://brat.nlplab.org/standoff.html).**  
You may want to convert the format into some internal representation for training models; however, how you do that is up to you, i.e. you do not have to use IOB format like in Assignment 2. 

**Further Notes**:
- At training time, you you will be provided with plain text documents and `.ann` files with keyphrases and relations
- At test time, you will be provided with plain text documents and `.ann` files **with keyphrases only**. This is because your task is to predict relations.
- The evaluation script is agnostic to the order of relation triples and relation ids, but should preserve the ids of the keyphrases that will be used in the predicted relations. The evaluation scripts requres the entity annotations to be present as well in the prediction file.

## <font color='blue'>Task 1.1</font>: Develop and Train a Relation Extraction Model with Gold Keyphrases

In this task, you develop a relation extraction model and apply it to the ScienceIE dataset.
As input to it, at test time, you will have the plain input texts as well as `.ann` files containing gold (i.e. correct) keyphrase annotations. The output should be `.ann` files containing relations between those keyphrases (you should include the keyphrase annotations in the output file as well).

A test input/output example is given in folders `data/scienceie/test/`,`data/scienceie/test_pred/`.

There are no strict requirements for how to design this model. You are expected to use the knowledge you have gathered throughout this course to design and implement this model. 

You are welcome to re-use existing code you might have written for other assignments as you see fit.

You are free to implement your solution in either PyTorch or Tensorflow, but if you are not sure where to start, we recommend looking at the [Keras API](https://keras.io) which is [integrated into Tensorflow 1.14.0](https://www.tensorflow.org/beta/guide/keras/overview?hl=en).

### Module imports

To highlight the important parts of the assignment, most of the functions are written in modules in 'Extra_files/modules'. These are imported below.

In [ ]:
### General modules
import pprint
from gensim.models import fasttext
from gensim.models import KeyedVectors
from copy import deepcopy
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from pickle import load, dump
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, Input, Concatenate, Conv1D, Flatten, Dropout, LeakyReLU
from tensorflow.keras.losses import categorical_crossentropy  # , sparse_categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
pp = pprint.PrettyPrinter(compact = True).pprint
tf.logging.set_verbosity(tf.logging.ERROR)


### Our own modules
from Extra_files.modules.load_and_save import load_scienceie, save_to_ann, reformat_to_save
from Extra_files.modules.rules import _add_rules
from Extra_files.modules.error_analysis import plot_confusion_matrix, get_dataframe, plot_correct_labels
from Extra_files.modules.DataPreparation import entityLocator, addPOStoDic, addLemmatoDic, inputPair
from Extra_files.modules.WordEmbedder import WordEmbedder
from Extra_files.modules.MasterVocab import MasterVocab
from Extra_files.modules.scaling import downscale, upscale
from Extra_files.models.dummy_model import _sebastians_dummy_model
from Extra_files.modules.DataSeq import trainable_emb
from Extra_files.modules.helpers import f1_score_ala_calc_measures, HiddenPrints
from Extra_files.modules.Dynamic_pooling import Dynamic_max_pooling
from Extra_files.models.fasttext_cnn2 import fasttext_cnn_model
from Extra_files.models.fasttext_cnn_UMLS import fasttext_cnn_UMLS_model
from Extra_files.modules.Encoding_and_formatting import enc, to_tf_format


# Data pre-processing

To identify relations between entities, the input data goes through several pre-processing steps:

##### Conversion to IOB format
First, the .ann and .txt files are converted to IOB format, as we need to identify the entities and the sentences between these for the later modelling. In this conversion, several entites have to be removed, as entities overlap. In this case, the longest entities have been kept. This also leads to a slight loss in the number of relations, as some of these referes to entities, which are removed due to overlap.

##### Removing references
In several texts, their are references to other text ([12], [13-14] etc.). In most cases however, these do not add any value to identify relations, and this is only noise for our modelingl. Furthermore, references are often in the sentence between hyponyms and hypernyms. An example, where the references can complicate the relation detection can be seen in the first row below, where a reference is in between to entities with a hypernym relation.

##### Concatenating 'e.g.' and 'i.e.'
Multiple entities with relations a mentioned with 'XXXX, e.g. YYYY' and 'XXXX, i.e. YYYY'. An example of both of these cases can be seen in line 2 and 3 in the table below. If the tokens are split as 'e', '.', 'g' and '.', it may very well be more difficult for the model to learn, than if it is a single token. Therefore these are concatenated to a single.
<br>

<img src="Extra_files/figures/Filter_examples.png">

#### Transformation to model input
To train the model, every entity-pair, and the sentence between these are extracted. This gives us $\frac{n_i \cdot (n_i-1)}{2}$ data points per file (where $n_i$ is the number of entities in the file).

#### Downscaling
With the format as above, the dataset is severely imbalanced with 99% of the entities having no relation. This would make it almost impossible for the model to train. Therefore, for all entities mentioned in different sentences, no relationship is assumed, and these datapoints are removed. This reduces the size of the data sets with roughly 80%, and in the train data, this corresponds to removing 45785 data points with no relations, while only accidantly removing 5 datapoints with a relation.

#### Upscaling
As the dataset after imbalance is still significantly biased (with ~95% of datapoints having no relations), the datapoints with relations is artificially upscaled with a factor 10. This will hopefully get the model to learn the features for relations faster. The upscaling will naturally only be done on the training set, and the development set will then be used to rescale the probabilities again. <br>
The distribution of the labels before downscaling, after downscaling and after upscaling of relations can be seen in the figure below.


<img src="Extra_files/figures/Label_dist.png">


## Load_scienceie
This function handles the three first pre-processing steps:
- IOB conversion
- Removing references
- Concatinating 'e.g.' and 'i.e.'

In [ ]:
print("Loading dev_data...")
dev_data = load_scienceie(datadir=join(_snlp_book_dir, "data", "scienceie", "dev"))
print("Loading train_data...")
train_data = load_scienceie(datadir=join(_snlp_book_dir, "data", "scienceie", "train"))

### Spacy

We download the spacy 'en' model to utilize its "Part-of-Speech" tagging and lemmatization for the words in our data. The data at hand, and in particularily the entities, are generally very scientific. This poses a challenge for the *'vanilla'* spacy package. There is an alternative package called scispacy which should perform better. Unfortunately, installing scispacy and downloading a suitable scientific model takes too much of our allowed running time. <br>
In the cell below, we import and install our spacy model: <br>
*NOTE:* We realized too late that you will install scispacy yourselves, and do not have time to fix this issue. Too bad...

In [ ]:
%%capture
!python -m spacy download en
import spacy
nlp = spacy.load("en")
nlp.tokenizer = nlp.tokenizer.tokens_from_list

### Transform to indices and Vocabularies
We generate three different vocabularies - one for words and lemma, one for Part-of-Speech and one for Entities. These vocabularies are all stored within our class "MasterVocab". This allows for seamless passing and controlling the vocabularies throughout the entire notebook.<br> The "MasterVocab" class contains a method that generates the three vocabularies from data and stores them internally. Additionally, the class has methods to map the raw X data input into indices via the respective vocabularies. Using an embedder class which will be introduced in the next cell, it can also return word embeddings for both the words and lemmas for our X data.

In [ ]:
# Generate the Vocab classes. 
if True:
    vocab = load(open('Extra_files/resources/vocab.pkl', 'rb'))
if False:
    vocab = MasterVocab(max_vocab_size = 4092)
    vocab.generateVocabularies(train_data, nlp_language)
    dump(vocab, open('Extra_files/resources/vocab.pkl', 'wb'))

### Model for FastText embedding

The words and lemmas of our data is transformed into word embeddings via FastText. This has several benefits. It allows us to reduce the entire vocabulary of words into vectors of length 300 - the standard vector dimension for FastText. An alternative to this would be to learn our own embedding in a Tensorflow layer. This would however result in an serious increase in the number of trainable parameters, which both scales the training time significantly and leads us vulnerable to overfitting of the embeddings. <br>

**Our FastText Model**<br>
We use the 'wiki-news-300d-1M' FastText model which contains vectors for a vocabulary of 1 million words. Naturally, we cannot use this model in its raw form due to our limitations on memory (2GB). Instead, we load in a smaller subset of that FastText model with a limit of vocabulary of 15,000 words. The scientific nature of data has the consequence that a lot of our tokens cannot be found in such a limited FastText model.<br>
To alleviate this issue, we introduce another FastText model with a vocabulary of 350,000 words. We then check each token in our vocabulary from data. If a specific token is found in the rich FastText model, but not the limited one, we add the token to our limited FastText model from the rich FastText model. This approach means that we eventually add roughly 7,500 extra words to the vocabulary of our limited FastText model - such that it contains a total of 22,500 words. This results in a relatively small FastText model with an impressive 89% coverage of tokens in vocabulary from data. We save the final FastText model and discard the large model with 350,000 words to free up memory.
<img src="Extra_files/figures/FastTextModel.png">

**Reducing dimensionality of FastText**<br>
The FastText models all return word embeddings of a dimension of 300. This means that each token in our data would have a dimension of 600 from embedding the word and lemma alone. Even though we have quite a lot of data to work with, such a huge dimensionality increases the risk of overfitting, and on top of that it can introduce memory issues.<br>
We reduce the dimensionality from 300 to 50 via Principal Component Analysis (PCA). The PCA allows us to reduce the dimensionality effectively, such that we do not lose more information than necessary. The illustration below shows the explained variance ratio as a function of components. Our choice of 50 components capture roughly 65% of the variability in our embeddings.

<img src="Extra_files/figures/PCA_pic.png">

**The Embedding class**<br>
We implement the class "WordEmbedder" which can build, save and load the FastText model as described above. The class has a method 'getEmbedding' which returns the word embeddings if the token is found in our FastText model, otherwise it returns an array of zeros before the PCA transformation. Furthermore, we append an indicator of 0 if the token was found in the model or 1 if it did not. This indicator is appended after the PCA transformation to avoid it being distorted. This principle is illustrated below:

<img src="Extra_files/figures/PCAFT.png">

The WordEmbedder uses the method "loadEmbedder" which loads in the FastText Model along with the pre-trained PCA.

In [ ]:
## Load the English Embedder and PCA
engbedder = WordEmbedder()
engbedder.loadEmbedder()

The Figure below shows the full pre-processing procedure: <br>

<img src="Extra_files/figures/Preprocessing.png">


**Relative distances between entities**<br>
To help the model incorporate the importance of the words in the sentences, we added the distance from every token to both the first entity and second entity to the input.

The final format of every pre-processed token is illustrated below. This is the input that our model receives.

<img src="Extra_files/figures/ModelInput.png">

### The function 'dataX_Y_format'
The function below handles:
- Reshaping to train data format
- Downscaling
- Upscaling train data
- Adding lemmas
- Adding POS
- Adding Principal Components of FastText embeddings

In [ ]:
### I have split the functions but made it backwards compatible
def dataX_Y_format(data, indices = False, embeddings = False, embedder = engbedder):
    data_m_XY = dataX_Y_format_no_index(data)
    data_m_XY = downscale(data_m_XY)
#     data_m_XY = upscale(data_m_XY) # Only for train
        
    if indices or embeddings:
        if embeddings:
            data_m_XY = transform_to_embeddings(data_m_XY, embedder)
        else:
            data_m_XY = transform_to_indices(data_m_XY)
        #end-if
    return data_m_XY

# Only without index
def dataX_Y_format_no_index(data, nlp_language = nlp):
    
    metadata = []
    data_X = []
    data_Y = []
        
    for txt, file in data.items():
        addPOStoDic(file, nlp_language)
        addLemmatoDic(file, nlp_language)
        entities = entityLocator(file)
        
        #Labels
        labels = {}
        labels['Synonyms']= [(rel[2],rel[3]) for rel in file['relations'] if rel[1] == 'Synonym']
        labels['Hyponyms'] = [(rel[2],rel[3]) for rel in file['relations'] if rel[1] == 'Hyponym']
        
        # create a 'stair' of combinations
        
        for i in range(len(entities)):
            for j in range(i+1, len(entities)):
                
                #Extrac X and annotation names
                ann_names = (entities[i][0],entities[j][0])
                ann_names_reverted = (entities[j][0],entities[i][0])
                
                metadata.append( (txt,entities[i][0],entities[j][0]))
                
                xdata = inputPair(entities[i], entities[j], file, nlp_language)
                data_X.append(np.array(xdata))
                
                ### Extract label
                if ann_names in labels['Hyponyms']:
                    data_Y.append('Hyponym')
                elif ann_names_reverted in labels['Hyponyms']:
                    data_Y.append('Hypernym')
                elif ann_names in labels['Synonyms'] or ann_names_reverted in labels['Synonyms']:
                    data_Y.append('Synonym')
                else:
                    data_Y.append('NONE')
                    

    #We need it all in a single object
    data = {'metadata': metadata, 'data_X': data_X, 'data_Y': data_Y, 'Vocab': vocab}
    
    return data

#Transform to indices
def transform_to_indices(data_m_XY):
    '''
    Inplace transform of words to indices
    
    Uses 'vocab' from global env.
    '''
    for idx in range(len(data_m_XY['data_Y'])):
        data_m_XY['data_X'][idx] = vocab.transformX_toIndex(data_m_XY['data_X'][idx]) 
    
    return data_m_XY

# Transform to Embeddings & indices
def transform_to_embeddings(data_m_XY, embedder):
    '''
    Inplace transform of words to embeddings and indices
    
    Uses 'vocab' from global env.
    '''
    for idx in range(len(data_m_XY['data_Y'])):
        data_m_XY['data_X'][idx] = vocab.transformX_toEmbeddings(data_m_XY['data_X'][idx], embedder) 
    
    return data_m_XY

#### Load pre-transformed data
The data-transformations above take a while to run, so train and dev data has been pre-transformed leaving only the test data to be transformed runtime. The code below reads the pre-transformed data.

In [ ]:
if True:
    data_m_XY_dev = load(open('Extra_files/resources/dev_emb.pkl', 'rb'))

    
    dev_X = data_m_XY_dev['data_X']
    dev_Y = data_m_XY_dev['data_Y']
if False:
#     data_m_XY = load(open('Extra_files/resources/train_emb_up10.pkl', 'rb'))
    
    data_m_XY = dataX_Y_format(train_data,indices = True) 
    dump(data_m_XY, open('Extra_files/resources/train_emb.pkl', 'wb'))
    data_m_XY_dev = dataX_Y_format(dev_data, embeddings = True)
    dump(data_m_XY_dev, open('Extra_files/resources/dev_emb.pkl', 'wb'))
    
    train_X = data_m_XY['data_X']
    train_Y = data_m_XY['data_Y']

The following code is used to ensure data is only transformed once to reduce runtime:

In [ ]:
train_keys = frozenset(train_data.keys())
dev_keys = frozenset(dev_data.keys())
pre_transformed_data = {}
pre_transformed_data_UMLS = {}

distr = {True: pre_transformed_data_UMLS, False: pre_transformed_data}

def cached_data_formatter(data, UMLS=False):
    """
    Load pretransformed data if available.
    return:
        formatted data (incl metadata), and a tensorflow-friendly format
    """
    new_keys = frozenset(data.keys())
    
    if new_keys == train_keys:
        if not UMLS:
            return data_m_XY, trainable_emb(train=True, up=10)
        else:
            return data_m_XY, trainable_emb(train=True, UMLS=True)
        
    elif new_keys == dev_keys:
        if not UMLS:
            return data_m_XY_dev,  trainable_emb(train=False)
        else:
            return data_m_XY_dev, trainable_emb(train=False, UMLS=True)
        
    elif new_keys in distr[UMLS].keys():  # new data
        return  distr[UMLS][new_keys]
    
    else:
        full_formatted = dataX_Y_format(data, embeddings=True)
        if UMLS:
            UMLS_vals = []
            data_m_XY2 = dataX_Y_format(data, indices = False) 
            data_m_XY2 = add_UMLS(data_m_XY2)
            tmp = [str(x) for x in data_m_XY2['metadata']]
            tmp2 = data_m_XY2['UMLS']
            for i, met in enumerate(full_formatted['metadata']):
                idx = tmp.index(str(tuple(met)))
                UMLS_vals.append(tmp2[idx])
            
            ret = full_formatted, to_tf_format(enc, full_formatted['data_X'],  None, UMLS_vals)
            distr[UMLS][new_keys] = ret
            return ret 
        else:
            UMLS_vals = None
            ret = full_formatted, to_tf_format(enc, full_formatted['data_X'],  None, None)
            distr[UMLS][new_keys] = ret
            return ret 


<br>

# The model

### Model architecture
We train a CNN model on the sentences with the model-input above. The architecture is as follows:
- The POS and IOBtag is embedded in an embedding layer.
- The embedded data is concatenated with the remaining input, i.e. distances and FastText embeddings for tokens and lemmas.
- Next, this is used as input in 1D convolutional layer with a 'relu' activation function.
- On top of this, a dynamic max pooling layer reduces the size to a fixed size.
- In the end, a dense layer with a 4-dimensional output array predicts the probabilities of `NONE`, `Synonym`, `Hyponym` and `Hypernym` with a 'softmax' activation function.


#### Dynamic max pooling

Instead of padding the sentence prior to input, the datapoints used as input have different length. The convolution layer thus also has varying output length, and the length is then fixed in the dynamic max pooling layer. An example of the dynamic max pooling layer with a fixed output size of 5 can be seen below.

<img src="Extra_files/figures/DynMaxPool.png">

<br>
### The 'create_model', 'train_model' and 'make_prediction' functions

In [ ]:

def create_model(train_data, dev_data):
    """
    Return an instance of a relation extraction model defined over the dataset.
    Args:
        train_data: the training data the relation extraction detection model should be defined over.
        dev_data: the development data the relation extraction detection model can be tuned on.
    Returns:
        a relation extraction model
    """
    
    if True:
        return fasttext_cnn_model()
    
    else:
        ## PARAMETERS

        # Embeddings
        entity_vocab_size = 20
        pos_vocab_size = 20

        # word_emb_dim = 50
        entity_emb_dim = 10
        pos_emb_dim = 10

        # Conv1D
        filters = 200
        conv_kernel_size = 1
        conv_strides = 1

        # Pooling
        dyn_out_len = 100

        ## LAYERS

        # Input
        tokens = Input(batch_shape=(1, None, 51))
        relpos1 = Input(batch_shape=(1, None, 1))
        relpos2 = Input(batch_shape=(1, None, 1))
        entity = Input(batch_shape=(1, None))
        pos = Input(batch_shape=(1, None))
        lemma = Input(batch_shape=(1, None, 51))

        # Embedding
        entity_emb = Embedding(entity_vocab_size, entity_emb_dim)(entity)
        pos_emb = Embedding(pos_vocab_size, pos_emb_dim)(pos)

        # Base-model
        x = Concatenate(axis=2)([tokens, relpos1, relpos2, entity_emb, pos_emb, lemma])
        x = Dropout(0.5)(x)
        x = Conv1D(filters=filters, kernel_size=conv_kernel_size, strides=conv_strides, padding='same', activation='relu')(x)
        x = Dynamic_max_pooling(filters, dyn_out_len)(x)
        x = Flatten()(x)
        x = Dropout(0.5)(x)
        x = Dense(4, activation='softmax')(x)  # 4 == n_classes in data (i.e. NOT a parameter)

        ## Create model
        model = Model(inputs=[tokens, relpos1, relpos2, entity, pos, lemma], outputs=x)

        model.compile(loss=categorical_crossentropy,  # for onehot, for idx: sparse_categorical_crossentropy
                      optimizer=Adam(),
                      metrics=['accuracy'])

        return model

def train_model(model, train_data, dev_data):
    """Train a relation extraction model on the given dataset.
    Args:
        model: The model to train
        data_train: The dataset to train on
        dev_data: the development data the relation extraction detection model can be tuned on
    """
    

    
    if True:
        # Load pre-trained
        filters = 200
        dyn_out_len = 100
        path = os.path.join(os.getcwd(),'Extra_files/resources/fasttext_cnn_model_{}_{}/'.format(filters, dyn_out_len))
        path = os.path.join(path, list(os.listdir(path))[0], 'final_save')
        model.model.load_weights(path)
    else:
        callbacks = [ModelCheckpoint(filepath, monitor='val_loss', save_best_only=False, save_weights_only=True),
                    EarlyStopping(patience=15, restore_best_weights=True)]

        train_set = to_tf_format(enc, train_X, train_Y)
        dev_set = to_tf_format(enc, dev_X, dev_Y) 
        epochs = 100
        model.model.fit_generator(generator=train_set, steps_per_epoch=len(train_set), epochs=epochs,
                              callbacks=model.training_callbacks(), verbose=1,
                              validation_data=dev_set, validation_steps=len(dev_set),
                              max_queue_size=100, workers=8, use_multiprocessing=True)

        
    
    return

def make_predictions(model, data, return_raw = False, threshold=0.4):
    """Makes predictions on a list of instances
    Args:
        model: The trained model
        data: The dataset to evaluate on
    Returns:
        The model's predictions for the data.
        If 'return_raw = True, it returns 'data_dict' (see below), else it returns reformatted data ready for saving.
    """
    
    
    local_data_m_XY, tf_formatted = cached_data_formatter(data)
    raw_preds = model.model.predict_generator(tf_formatted, workers=1, use_multiprocessing=False)  # probs
    if threshold is not None:
        raw_preds[:,2] += threshold
        
    predictions = list(np.squeeze(enc.inverse_transform(raw_preds)))  # human-readable

    ### Save in correct format
    local_data_m_XY['data_Y'] = predictions
    if return_raw:
        return local_data_m_XY
    else:
        #data = deepcopy(data)   ##### DOES NOT WORK ATM - USING GLOBAL VARS
        save_format = reformat_to_save(local_data_m_XY)
        for txt in data:
            data[txt]['relations'] = save_format.get(txt,[]) #We might have txt's with no data after downscaling

        return data


In [ ]:
# Create model
model = create_model(train_data, dev_data)
train_model(model, train_data, dev_data)


### Rescaling the probabilities

As we upscaled the number of relations during training, the model learned a different label distribution than the correct. To account for this, we will in the prediction fase add a probability to the `NONE` label (corresponding to subtracting a probability from the three upscaled labels).

The perentage addition will be chosen by tuning on the Micro F1-score, which is the score that will be used in the final evaluation. The F1-score is calculated with 'CalculateMeasure' function. The code can be seen below.

```python
import sys,os

## Calculate F1-scores
thresholds = np.arange(-90,91,1) * 0.01
f1_scores = []
for threshold in thresholds:
    dir_dev = join(_snlp_book_dir, "data", "scienceie", "dev")
    data_pred = make_predictions(model, dev_data,threshold = threshold)
    dir_pred = join(_snlp_book_dir, "data", "scienceie", "dev_pred")
    save_to_ann(data_pred, dir_pred)
    with HiddenPrints():
        metrics = calculateMeasures(dir_dev, dir_pred, "keys")
    
    f1_scores.append(metrics['overall']['f1-score'])
    
## Plot
fig, ax = plt.subplots(figsize=(8,5.33))
ax.plot(thresholds,f1s)
labs = np.arange(-1,1,0.25)
ax.set_xticklabels(['{:,.0%}'.format(x) for x in labs])
ax.set_title('Micro F1-scores when changing the predicted probability of NONE')
ax.set_xlabel('Percentage addition to NONE')
ax.set_ylabel('F1-score')
plt.show()
```

This results in the Figure:
<img src="Extra_files/figures/F1_tuning.png">

We see, that by adding 10-45% to the `NONE` label before choosing the label with highest probability, we will maximize the f1-score and get around 28%. The reason why it stretches from 10% to 45%, is that the F1-score for the `Hyponym` label is maximized around 10%, while the F1-score for `Synonym` is maximized around 45%. This is simply due to different distributions in the train and dev set. As the total F1-score is maximized at 40%, this will be chosen in the final prediction. 

### Confusion matrix

To investigate the accuracy within each group, we plot the confusion matrix. <br>
NOTE: This matrix is calculated on model input data, i.e. after removing overlapping relations and removing entities in separate sentences. It is therefore different from the 'calculateMeasures' function.

In [ ]:
#Make predictions
y_base = make_predictions(model, dev_data, return_raw = True)['data_Y']
y_true = dev_Y

### Create plot
fig, ax1 = plt.subplots(figsize=(14,3.8))
ax1 = plot_confusion_matrix(y_true, y_base, n_labels=None,normalize=True,ax = ax1)
plt.show()

We see, that we are good at detecting `Synonym`, not so good at `Hypernym`and especially bad at `Hyponym`. It is worth noting, that more `Hyponym` are predicted to be `Synonym` than correct. This indicates, that these labels are especially difficult to distinguish, and as we have more `Synonym` labels than `Hyponym` labels (see pie chart above), we predict `Synonym` when in doubt.

### The 'calculateMeasures' function on dev set

To ensure everything works as expected, we run the 'calculateMeasures' function on the dev set:

In [ ]:
# Use this cell to test on the dev set
data_train = load_scienceie(datadir=join(_snlp_book_dir, "data", "scienceie", "train"))
dir_dev = join(_snlp_book_dir, "data", "scienceie", "dev")
data_dev = load_scienceie(datadir=dir_dev)

model = create_model(data_train, [])
train_model(model, data_train, [])

data_pred = make_predictions(model, data_dev)
dir_pred = join(_snlp_book_dir, "data", "scienceie", "dev_pred")
save_to_ann(data_pred, dir_pred)

calculateMeasures(dir_dev, dir_pred, "keys") # this will only evaluate the correctness of relations

In [ ]:
from time import time
p0 = time()
# DO NOT MODIFY THIS CELL! It will evaluate your model on an unseen dataset!
#shutil.rmtree(join(_snlp_book_dir, "data", "scienceie", "test_pred")) # clean after previous

data_train = load_scienceie(datadir=join(_snlp_book_dir, "data", "scienceie", "train"))
data_dev = load_scienceie(datadir=join(_snlp_book_dir, "data", "scienceie", "dev"))

model = create_model(data_train, data_dev)
train_model(model, data_train, data_dev)
data_test = load_scienceie(datadir=join(_snlp_book_dir, "data", "scienceie", "test"))
data_pred = make_predictions(model, data_test)
dir_pred = join(_snlp_book_dir, "data", "scienceie", "test_pred")
save_to_ann(data_pred, dir_pred)

dir_gold = join(_snlp_book_dir, "data", "scienceie", "test_gold")
calculateMeasures(dir_gold, dir_pred, "keys") # this will only evaluate the correctness of relations

print((time()-p0) / 60)

## <font color='red'>Assessment 1.1</font>: Correctness of the implementation (20 pts)

We assess if your code implements a correct relation extraction model (10 points):

* 0-5 pts: the model does not run correctly or does not constitute a relation extraction model
* 5-10 pts: the model correctly implements the requirements

Additionally, we will assess how well your model performs on an unseen test set (10 points):

* 0-5 pts: performance worse than a simple baseline model
* 5-10 pts: performance better than a simple baseline model

## <font color='blue'>Task 1.2</font>: Describe your Approach

Enter a maximum 500 words description of your model developed in Task 1.1, its architecture, and the way you trained and tuned it. Motivate your choices, describing potential benefits and downsides.

# Answer to Task 1.2
Here we will discuss the motivation of our choices and reflect on these. As we are very limitted in the word count, the model architecture - both the actual model and the preprocessing - is described in rich detail above, and it is assumed, that the architecture is already comprehended.

##### Embeddings
We choose to use pre-trained word embeddings to keep the number of trainable parameters low. We do not believe that our data is rich and vast enough to train meaningful embeddings without serious overfitting. On top of that, we chose to restrict ourselves to a embedding vector length of 50 via PCA. This is vital when considering that we use both token and lemma embeddings.<br>
Additionally, we include two drop-out layers and early-stopping to further alleviate the risk of overfitting. This is all motivated to avoid overfitting.<br>
<br>
Our model trains embeddings on POS and IOB-tags with a dimension smaller than that of the POS/Entities vocab size. This allows our model to fit these tags more closely to our data, while keeping the number of parameters low as a consequence of the small embedding dimension. <br>

##### CNN versus LSTM
To model the dependence between the tokens, we decided to use a CNN instead of e.g. an LSTM/bi-directional LSTM for several reasons. Firstly, the 2nd assignment showed, that the bi-LSTM could be very difficult to tame. Secondly, in the SemEval 2017 Task 10, a model with CNN architecture achieved the highest performance - a great motivation to try this architecture ourselves.

##### Variable length input
As described above, the input to the model is the tokens between (inclusive) 2 enteties. This yields variable length inputs for the model. One approach to this issue would be a model that allows variable length inputs yet deliver fixed sized output, such as an LSTM only returning the final output, however such models are scarce. A second would be padding all inputs to some large fixed size, somehow cropping all inputs longer than this large fixed size, but this yields a more padding that one could wish, and issues if some inputs are particularly large. We decided to opt for a third solution, trying to mitigate particularly the issues of the second solutions. Our third solution was a dynamic max pooling layer.

In this layer, a fixed output size is decided on beforehand, simiarly to when padding, but this time, it does not need, and should not, be generally larger than the inputs. It then works, as previously stated, by padding any short input, and by computing a variable sized non-overlapping max-pooling operation on longer inputs. *EXAMPLE*
With this solution, a new hyperparameter has been introduced, but it allows combining a variable-to-variable length layer, such as a convolution in our case, with a fixed sized layer, e.g. a dense layer, to attain the strength of both.

### Model Tuning and training
##### Upscaling and downscaling
The downscaling, i.e. the removal of examples where entities are far from each other, and upscaling, i.e. the repetition of examples where the relation between entities is not `NONE`, both, as described above, are implemented to deal with the severe class imbalance. If attempting to train a model based on accuracy and a class imbalance with more than 99%, it is difficult to achieve any other convergence than towards classifying all examples as the most observed label, as nearly no other cases will be represented in the learning process.

##### Tuning
After training the up- and downscaled data, ideally the model will in fact be able to have accurate predictions on both the most common label as well as the others. However, the final evalution criteria is a F1-score, so it is possible that the most accurate predictions are not the most optimal. This is dealt with, by not immidiately choosing the label predicted as the most likely by the model, but, as described above, by adjusting the predicted probability of `None`, and only then choosing the label with the highest adjusted probability. Above, it is demonstrated how adjusting towards picking `None` more often than the model predicts, yields better F1-score.                                         

## <font color='red'>Assessment 1.2</font>: Modelling Choices and Motivation (10 pts)


Finally, we assess your modelling design choices and how you motivated them, which you summarised in the above cell (10 points):

* 0-5 pts: the model design choices do not show high levels of creativity, e.g. re-using code from the lecture out of the box; and they are not moviated well
* 5-10 pts: the model design choices show high levels of creativity, e.g. combining different things learned throughout the course, models inspired by further reading, etc.; and they are motivated well

## <font color='blue'>Task 2</font>: Relation Extraction with Weak Supervision

In this task, the goal is to improve the performance of your model developed in Task 2 by obtaining more automatically labelled training data using a weak supervision approach. You are not required to change the relation extraction model architure, i.e. it is fine to re-use the one from Task 1, but instead, the requirements are to implement one or more weak supervision strategies.

Some possible weak supervision methods for relation extraction will be introduced in the lecture Week 43 (https://github.com/copenlu/stat-nlp-book/blob/master/chapters/relation_extraction_slides.ipynb); the following blog post also serves as a good introduction to this topic: https://hazyresearch.github.io/snorkel/blog/ws_blog_post.html

For this task, you are not confined to the training data provided to you, but you are welcome to obtain additional unlabelled datasets and automatically label them using weak supervision methods. 

The general setup will otherwise be the same as for Task 1:
As input to it, you will have the plain input texts as well as `.ann` files containing gold (i.e. correct) keyphrase annotations. The output should be `.ann` files containing relations between those keyphrases.

**Important notes**:
- You must provide code for the functions below. 
- If running them on the full dataset exceeds the 10 minute limit, you are welcome to additionally provide a line of code that (down)loads the already weakly annotated data.
- The maximum file size for weakly annotated data may not exceed 1GB.

A test input/output example is given in folders `data/scienceie/test/`,`data/scienceie/test_pred/`.

## Unified Medical Language System

The Unified Medical Language System (UMLS) contains multiple Knowledge Sources within the medical industry. From the UMLS we will use the `Semantic Network` dataset (https://semanticnetwork.nlm.nih.gov/). This dataset contains 133 semantic types and 54 semantic relationships. <br>
The primary link between the semantic types is the ”isa”, which establishes a hierarchy relationship between the semantic types. The link "X 'isa' Y" establishes that X is a hyponym of Y.

This dataset contains 614 hyponym relations among the 133 semantic types.

This dataset will be incorporated in the model by adding an extra feature to the dense layer.For every entity pair in the dataset, these will be lookup up in the UMLS table. If the first entity pair had the "isa" relation to the second pair, the feature will have the value 1. If it was the other way around, it has value of -1, otherwise 0. The dense layer should then be expected to tune the parameters accordingly.

In [ ]:
## Load Hypernyms from UMLS
from Extra_files.modules.UMLS import load_UMLS, add_UMLS

relations = load_UMLS()
len(relations)

## Domain knowledge

### Abbreviations as synonyms
In scientific context, new concepts are often introduced by writting the full name and an abbreviation in a parenthesis afterwards as XXXX (YYYY), see e.g. the example with UMLS above. This is implemented as a strong rule, such that any entity followed by another entity in a parenthesis written only as capital letters will be marked as synonyms. Below is an example of a synonym entity pair.

<u> Example </u> <br>

- S0010938X13002187:    &nbsp; &nbsp; &nbsp;  "... a **thermogravimetric balance** (**TGA**) ... "


### Link lists
Hyponyms of a general concepts are often mentioned in lists, where several examples of subsets of the broader categories are given. Two different examples can be seen in the table below.<br>

To account for this, entities are 'linked' together if they are mentioned in a list. These lists are defined as having the three formats:

- X, Y, Z
- X, Y and Z
- X, Y or Z

Whenever at least three entities are mentioned after each other (with the last being on one of the forms [", Z", "and Z", "or Z"]), these are linked together as a list. When they are linked together in a list, the following rules are applied:

1) None of them can have relation to each other.<br>
2) None of them can be synonyms.<br>
3) If one of them is Hyponym-of or Hypernym-of an entity outside, they all are.<br>

Below are example of entities mentioned in lists, where these are all hyponyms of an other entity mentioned before.

<u> Examples </u> <br>

- S0045782513000479: &nbsp; &nbsp; &nbsp; "... their applications to problems of various **areas** are given in [22,23], for example **shape offsetting**, **computing distances**,  &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; **photolithography development**, **seismic travel times**, etc. ..."<br>

- S0010938X15301189: &nbsp; &nbsp; &nbsp; "... **safety critical components**, such as **steam generator tubes**, **heater sleeves**, **pressurized instrument penetrations** and &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; **control rod drive mechanisms** [2,6,7]."

The domain knowledge is added in the post-processing step.




## Challenges

Alternative approaches to the weak supervised learning task would be to find text/data of similar characteristics as our current data i.e. scientific articles. We would annotate this new data and apply the bootstrap method between our labelled data and the unlabelled, but now IOB-annotated, new data. This approach requires, as a minimum, a decent NER (Named Entity Recognition) model to perform the classification of entites in the new data.<br>
Unfortunately, neither of us were able to develop a model of such quality to perform reliable annotation to use this approach for bootstrapping. As an alternative we decided to use UMLS.

In [ ]:
# You should improve this cell

def create_weak_model(train_data, dev_data, **args):
    """
    Return an instance of a relation extraction model defined over the dataset.
    Args:
        train_data: the training data the relation extraction detection model should be defined over.
        dev_data: the development data the relation extraction detection model can be tuned on.
        **args: any additional arguments needed, e.g. additional automatically labelled training data
    Returns:
        a relation extraction model
    """
    if True:
        return fasttext_cnn_UMLS_model()
    else:
         ## PARAMETERS

        # Embeddings
        entity_vocab_size = 20
        pos_vocab_size = 20

        entity_emb_dim = 10
        pos_emb_dim = 10

        # Conv1D
        conv_kernel_size = 1
        conv_strides = 1

        # Pooling

        ## LAYERS

        # Input
        tokens = Input(batch_shape=(1, None, 51))
        relpos1 = Input(batch_shape=(1, None, 1))
        relpos2 = Input(batch_shape=(1, None, 1))
        entity = Input(batch_shape=(1, None))
        pos = Input(batch_shape=(1, None))
        lemma = Input(batch_shape=(1, None, 51))

        UMLS = Input(batch_shape=(1, 1))

        # Embedding
        entity_emb = Embedding(entity_vocab_size, entity_emb_dim)(entity)
        pos_emb = Embedding(pos_vocab_size, pos_emb_dim)(pos)

        # Base-model
        x = Concatenate(axis=2)([tokens, relpos1, relpos2, entity_emb, pos_emb, lemma])
        x = Dropout(0.5)(x)
        x = Conv1D(filters=filters, kernel_size=conv_kernel_size, strides=conv_strides, activation='relu')(x)
        x = Dynamic_max_pooling(filters, dyn_out_len)(x)
        x = Flatten()(x)
        x = Concatenate(axis=1)([x, UMLS])  # ADD UMLS DIRECTLY TO DENSE INPUT
        x = Dropout(0.5)(x)
        x = Dense(4, activation='softmax')(x)  # 4 == n_classes in data (i.e. NOT a parameter)

        ## Create model
        model = Model(inputs=[tokens, relpos1, relpos2, entity, pos, lemma, UMLS], outputs=x)
        #     model.summary()

        model.compile(loss=categorical_crossentropy,  # for onehot, for idx: sparse_categorical_crossentropy
                      optimizer=Adam(),
                      metrics=['accuracy'])

        return model

def train_weak_model(model, train_data, dev_data,**args):
    """Train a relation extraction model on the given dataset.
    Args:
        model: The model to train
        data_train: The dataset to train on
        dev_data: the development data the relation extraction detection model can be tuned on
        **args: any additional arguments needed, e.g. additional automatically labelled training data
    """
    
    if True:
        # Load pre-trained
        filters = 200
        dyn_out_len = 100
        path = os.path.join(os.getcwd(),'Extra_files/resources/fasttext_cnn_UMLS_model_{}_{}/'.format(filters, dyn_out_len))
        path = os.path.join(path, list(os.listdir(path))[0], 'final_save')
        model.model.load_weights(path)
    else:
        train_set = trainable_emb(train=True, UMLS=True)  # load pickled
        dev_set = trainable_emb(train=False, UMLS=True)  # load pickled
        
        callbacks = [ModelCheckpoint(filepath, monitor='val_loss', save_best_only=False, save_weights_only=True),
                    EarlyStopping(patience=15, restore_best_weights=True)]

        epochs = 100
        model.model.fit_generator(generator=train_set, steps_per_epoch=len(train_set), epochs=epochs,
                              callbacks=model.training_callbacks(), verbose=1,
                              validation_data=dev_set, validation_steps=len(dev_set),
                              max_queue_size=100, workers=8, use_multiprocessing=True)


def make_predictions_weak(model, data, return_raw = False, threshold = 0.1):
    """Makes predictions on a list of instances. Can be the same as function developed in Task 1.
    Args:
        model: The trained model
        data: The dataset to evaluate on
    Returns:
        The model's predictions for the data.
    """
    
    
    local_data_m_XY, tf_formatted = cached_data_formatter(data, UMLS=True)
    raw_preds = model.model.predict_generator(tf_formatted, workers=1, use_multiprocessing=False)  # probs
    if threshold is not None:
        raw_preds[:,2] += threshold
        
    predictions = list(np.squeeze(enc.inverse_transform(raw_preds)))  # human-readable
    local_data_m_XY['data_Y'] = predictions
    local_data_m_XY = _add_rules(local_data_m_XY,data)
    
    if return_raw == True:
        return local_data_m_XY  
    else:
        save_format = reformat_to_save(local_data_m_XY)
        for txt in data:
            data[txt]['relations'] = save_format.get(txt,[]) #We might have txt's with no data after downscaling
        return data



In [ ]:
# Create model
model = create_weak_model(train_data, dev_data)
train_weak_model(model, train_data, dev_data)

### CalculateMeasures on dev set

In [ ]:
# Directories
dir_dev = join(_snlp_book_dir, "data", "scienceie", "dev")
dir_pred = join(_snlp_book_dir, "data", "scienceie", "dev_pred")


#Make predictions
y_true = dev_Y
y_weak = make_predictions_weak(model, dev_data, return_raw = True)['data_Y']
data_pred = make_predictions_weak(model, dev_data)

save_to_ann(data_pred, dir_pred)
calculateMeasures(dir_dev, dir_pred, "keys")

#Plot
plot_confusion_matrix(y_true, y_weak, n_labels=None,normalize=True)
plt.show()

In [ ]:
# DO NOT MODIFY THIS CELL! It will evaluate your model on an unseen dataset!
# shutil.rmtree(join(_snlp_book_dir, "data", "scienceie", "test_pred")) # clean after previous

data_train = load_scienceie(datadir=join(_snlp_book_dir, "data", "scienceie", "train"))
data_dev = load_scienceie(datadir=join(_snlp_book_dir, "data", "scienceie", "dev"))

model = create_weak_model(data_train, data_dev)
train_weak_model(model, data_train, data_dev)

data_test = load_scienceie(datadir=join(_snlp_book_dir, "data", "scienceie", "test"))
data_pred = make_predictions_weak(model, data_test)
dir_pred = join(_snlp_book_dir, "data", "scienceie", "test_pred")
save_to_ann(data_pred, dir_pred)

dir_gold = join(_snlp_book_dir, "data", "scienceie", "test_gold")
calculateMeasures(dir_gold, dir_pred, "keys") # this will only evaluate the correctness of relations

In [ ]:
dir_pred = join(_snlp_book_dir, "data", "scienceie", "test_pred")
dir_gold = join(_snlp_book_dir, "data", "scienceie", "test_gold")

dir_gold = join(_snlp_book_dir, "data", "scienceie", "test_gold")
calculateMeasures(dir_gold, dir_pred, "keys") # this will only evaluate the correctness of relations

## <font color='red'>Assessment 2.1</font>: Correctness of the implementation (20 pts)

We assess if your code implements correct weak supervision methods (10 points):

* 0-5 pts: the model does not run correctly or the methods implemented do not constitute weak supervision strategies
* 5-10 pts: the model correctly implements the requirements

Additionally, we will assess how well your model performs on an unseen test set (10 points):

* 0-5 pts: performance worse than a simple baseline model
* 5-10 pts: performance better than a simple baseline model

## <font color='blue'>Task 2.2</font>: Describe your Approach

Enter a maximum 500 words description of your weak supervision stategies developed in Task 2.1 and the way you trained and tuned them. Motivate your choices, describing potential benefits and downsides.

##### Domain knowledge
It is likely that the CNN will correctly classify relations when the distance between the entities is short. However, we experienced that the CNN has difficulty classifying relations in lists, as the distance from the entity and the last entities in the list is much longer. By linking them together via our list-rule, we are able to correctly classify relations in lists - given that we identify the first pair as a relation.<br>

The addition of the abbreviations simply came from an investigation of the dataset. Here we saw, that rouhgly 70% of the relations came from abbreviations. Although the CNN was is very likely to detect these relations, the potential loss of performance, should the CNN miss a relation, was so great that this was implemented as an insurance.

##### UMLS
The UMLS dataset was added to catch potential specific relations in the medical articles. This was hoped to contain some of the entities described in the scientific articles. We believe that the UMLS will alleviate the issue that may arise, when classifying relations between entities where the distance is relatively large.


##### Extra considered additions
Since the original dataset comes from articles from ScienceDirect, it would be ideal to have included more articles from here. However we did not find any way to download them except on PDF an manually convert them to txt files or by creating a special ScienceDirect Backfiles account. The same problem was encountered at similar journals.

Other knowledge bases and ontologies were investigated - i.e. downloading data from the BioPortal's API (http://data.bioontology.org/), the metathesaurus from UMLS (https://www.nlm.nih.gov/research/umls/licensedcontent/umlsknowledgesources.html) or XML data from DrugBank (https://www.drugbank.ca/releases/latest).

In the end, these were all discarded, as we blieve that the possible gain from this would be considerably small. As explained above, our NER models from assignment 2 were not of sufficient quality to perform the annotation task ourselves, hence we are limited to already annotated data. We were unfortunately not able to find any such data.

Alternatively we could look up patterns and entities from the train set directly in the corpora and use these matches for distant supervision or bootstrapping. However we belived that the noise from would be too big to extract much value from.

If we were to predict relations in common text, the WordNet would also have been very interesting. It is however based on common lexicon (see https://wordnet.princeton.edu/frequently-asked-questions), so it would unfortunately not be of much use in this scientific context.

##### Tuning
For the linked lists, a free parameter determines the length of list that should be linked via our list-rule, when linking the relations. The sentence "XXXX and XXXX" could e.g. be a lists of two entities. This parameter was tuned on the dev set with the f1 score similar to when tuning the threshold in problem 1.1. The optimal parameter for the minimal list was 3. With a length of 2, we caught too many NONE's and with more than three caught too few relations.

The UMLS information was included in the Dense layer and the parameter was tuned along with the rest of the network.

## <font color='red'>Assessment 2.2</font>: Modelling Choices and Motivation (10 pts)


Finally, we assess your modelling design choices and how you motivated them, which you summarised in the above cell (10 points):

* 0-5 pts: the model design choices do not show high levels of creativity, e.g. re-using code from the lecture out of the box; and they are not moviated well
* 5-10 pts: the model design choices show high levels of creativity, e.g. combining different things learned throughout the course, models inspired by further reading, etc.; and they are motivated well

## <font color='blue'>Task 3</font>: Comparison of relation extraction models

Reflect on the models implemented in Tasks 1 and 2. What worked and didn't work well, and how would you explain this? How and when does the performance differ between the models and why might that be? You are expected to perform a small error analysis on the development set in order to answer these questions.

In [ ]:
# First a dateframe is created for later use 
df_err_an = get_dataframe(dev_data, data_m_XY_dev, y_true, y_base, y_weak)

### Accuracy on different labels

In [ ]:
### Plot seaborn with labels
_ = plot_correct_labels(df_err_an)
plt.show()

We see, that both models are good at predicting synonyms, which indicates, that the base model is capable of catching the abbreviations. We find that the weak model is better at `Hypernym` and `Hyponym`, which shows, that the links might catch extra from the links. However we also see, that the `NONE` percentage decreases with the weak model, as this only adds extra relations. The percentage decrease is small, but as the `NONE` category is much larger, it is quite a bit in absolute values. This is why the addition in f1-score is so small, though the accuracy seems to increase a lot when examining the plot above.<br>

The higher detection of `Hyponym` in the weak supervision could also indicate that the UMLS catches a few `Hyponym`/`Hypernym` relations. This is unfortunately not the case, as subsequent examples will show. The 133 semantic types were not included in the entity pairs (only few times in individual entities).

### Confusion matrix

To investigate the wrong predictions, we plot the confusion matrices:

In [ ]:
import seaborn as sns
sns.reset_orig() #seaborn changes matplotlib default settings. 

fig, (ax1, ax2) = plt.subplots(1,2,figsize=(14,3.8))

fig.suptitle('Normalized confusion matrices',y=1.1,fontsize = 15)
ax1 = plot_confusion_matrix(y_true, y_weak, n_labels=None,normalize=True,ax = ax1)
ax1.set(title = 'Model w. weak supervision')
ax2 = plot_confusion_matrix(y_true, y_base, n_labels=None,normalize=True, ax = ax2)
ax2.set(title = 'Base model')
plt.show()

As mentioned earlier, we see that it is especially dificult to distinguish `Hyponym` from `Synonym`, and as we have more `Synonym` labels than `Hyponym` labels (see pie chart in the pre-processing section), we predict `Synonym` when in doubt. Otherwise the conclusions are the same as in the previous figure

### Examples

To get an indication of the correct and wrong predictions, we will investigate a few examples.

##### Correct Hypernyms

In [ ]:
pd.set_option('display.max_colwidth', -1)

df_err_an.loc[(df_err_an['True label'] == 'Hypernym') & 
          (df_err_an['Base label'] == 'Hypernym'), 
              ['file','Sentence between entities', 'Entity 1', 'Entity 2', 'True label', 'Weak supervision label']][:10]

Note that the CNN seems to catch some indicators for hypernym relations:

- 'e.g.' in line 1 and 2
- 'i.e.' in line 5
- 'such as' in line 6-10

This is quite impressive, and would also could also suggest, that concatenating 'i.e.' and 'e.g.' was a good choice

##### Wrong relations

Below are a few examples of wrong relations:

In [ ]:
pd.set_option('display.max_colwidth', -1)

df_err_an.loc[(df_err_an['True label'] == 'NONE') & 
          (df_err_an['Weak supervision label'] != 'NONE') &
          (df_err_an['Base label'] != 'NONE'), 
              ['file','Sentence between entities', 'Entity 1', 'Entity 2', 'True label', 'Base label']][9:15]

It is very transparant, why the CNN guessed many of these as hypernyms:

- 'such as' is present in line 1,2 and 6, which often indicated a Hypernym as we saw above
- 'i.e.' is present in line 3
- In line 4 the double appearance of 'as' might indicate a Hypernym relation in the CNN, because 'as' also appears in 'such as'

##### The difficulty of Hyponyms

Hyponyms were very difficult to identify for both models. This is due to the fact, that these sentences are very different from each other, and there are not a lot of indicators for these as is the case with 'such as' and 'e.g.' for the Hypernyms. The 5 examples below shows the point:

In [ ]:
df_err_an[(df_err_an['True label'] == 'Hyponym')][:5]

##### Differences in models

There are differences in the models as describe earlier, that the weak supervision model guesses more relations, both correct and wrong. However there are also many differences which are simply due to the randomness of the CNN. With the weak distribution implemented, the weak model should not really guess any `NONE` that the base model does not. However as we see below, this is also the case:

In [ ]:
df_err_an[(df_err_an['Base label'] != 'NONE') & (df_err_an['Weak supervision label'] == 'NONE')][:5]

### Finally: A challenge with the training
Often, the same entity is duplicated in the annotation file. This can create a serious issue, because only one of the duplicates is kept, and the rest are removed due to overlap. If a relation in the .ann file is tied to one of the duplicates which is removed, this relation is removed as well, and we are left with a data set indicating, that there is no relation between the entities.

This is the case in the four examples below, where the weak model actually predicts the true label according to the .ann file, but because of the duplicates, the data set reads the 'True label' as being NONE.

One example in the dev set is file S0010938X14002157. Here we have the following in the .ann file:

- T21	Material 1335 1352	Anionic sulphates
- T22	Material 1354 1373	anionic sulphonates
- T35	Material 1375 1405	alkoxylated alkylphenol resins
- T36	Material 1411 1443	polyoxyethylene sorbitan oleates
- T37	Material 1460 1471	surfactants
- T38	Material 1460 1471	surfactants
- T39	Material 1460 1471	surfactants
- T40	Material 1460 1471	surfactants
- R9	Hyponym-of Arg1:T21 Arg2:T40	
- R10	Hyponym-of Arg1:T22 Arg2:T39	
- R11	Hyponym-of Arg1:T35 Arg2:T38	
- R12	Hyponym-of Arg1:T36 Arg2:T37

However only T40 is saved, and T37-T39 is thrown out along with 3 of the relations. Below we see, that though the model actually guesses the label correctly, it is marked as being wrong:

In [ ]:
mask = [df_err_an['Entity 1'][idx] in ['Anionic sulphates','anionic sulphonates','alkoxylated alkylphenol resins','polyoxyethylene sorbitan oleates'] for idx in range(len(df_err_an))]
df_err_an.loc[(df_err_an['file'] == 'S0010938X14002157') &
          (df_err_an['Entity 2'] == 'surfactants') & 
          mask,['Sentence between entities','Entity 1', 'Entity 2', 'True label', 'Weak supervision label']]

## <font color='red'>Assessment 3</font>: Assess your explanation (20 pts)

We will mark the explanation along the following dimension: 

* Substance (20pts: well-designed error analysis, correctly explained reasons for performance differences between models)